## __SVM, 서포트 벡터 머신__
---

최적의 결정경계인 <b>초평면</b>을 찾아 분류/회귀를 하는 지도학습 기법입니다.  
복잡한 분류와, 소/중 규모의 데이터 세트에 적합합니다.  
스케일링이 필수적이며 평균을 0으로 맞춰주어야 합니다.  

* `sklearn의 .svm.SVC, .SGDClassifier(loss='hinge', alpha=1/(m*C))`  
 SVC가 더 빠르지만, SGD가 메모리 효율적이기 때문에 큰 데이터세트나 한정된 컴퓨팅 성능에 유리합니다.  


#### __0. 용어__


초평면: SVM이 찾고자 하는 목표입니다. 새로운 데이터를 분류/예측하는 기준이 됩니다.  
서포트 벡터: 초평면을 결정하는 각 클래스의 벡터(데이터)입니다.   
마진: 초평면과 서포트 벡터간의 간격입니다.  
마진 오류: 마진 사이에 있는 샘플로, 초평면에 의해 잘못 분류될 수 있는 것들입니다.  
> 마진 간격이 넓은 것이 바람직한 모델로 스케일링의 유무가 중요하게 작동합니다.  


### __1. 소프트마진 분류__


<b>하드마진 분류</b>란 초평면으로 완벽하게 구분되고, 마진오류 또한 없는 경우를 말합니다.  
하지만 데이터에 너무 의존적이며 이상치 하나에도 영향을 많이 받습니다.  

<b>소프트마진 분류</b>는 마진오류를 감수하면서 마진의 폭을 적절히 넓혀 일반화를 더 중시하는 방법입니다.  
규제를 가하면 마진이 넓어져 이를 구현할 수 있습니다.  



### __2. 비선형 분류__


4장의 다항회귀와 마찬가지로 데이터의 다항식 값을 추가하여 비선형 데이터를 처리할 수 있습니다.  
방식은 동일하지만, SVM에서는 많은 계산량을 보완해줄 방법이 있습니다.  

#### __2.0 커널트릭__
이 방법은 비선형 분류를 위한 실제 특성을 추가하지 않으면서 보다 적은 연산량으로 동일한 효과를 내는 기법입니다.  
> 자제한 방법은 아래 수식에서 정리하겠습니다.  

다양한 커널이 있지만, 비선형 분류에서는 다항식 커널, RBF 커널을 주로 사용합니다.  

* SVC의 커널을 linear로 주어 선형 SVM을 시행할 수 있지만, linearSVC가 최적화되어있어 훨씬 빠릅니다.

#### __2.1 다항식 커널__
다항식 값을 이용하여 계산하는 커널입니다.  

#### __2.2 RBF 커널__
각 샘플이 특정 <b>랜드마크 ($\ell$)</b>와 얼마나 유사한지 측정하는 <b>유사도 함수</b>의 값을 계산하는 커널입니다.  
유사도함수 RBf는 (Radial Basis Function, 방사기저함수) 아래와 같습니다. 
> $\phi_{\gamma}(\mathbf{x}, \ell)=exp(-\gamma \lVert \mathbf{x}-\ell \rVert ^2)$

이 함수는 0~1값을 가지는 가우시안 함수로, 랜드마크와 같은 위치의 경우 1입니다.  
감마가 높아지면 종 모양이 좁아지며 각 샘플에 더 민감하게 변화하기 때문에 낮은 감마를 규제와 비슷하게 사용할 수 있습니다.  

랜드마크를 여러개 지정하여 각 샘플을 각 랜드마크의 RBF로 변환, 고차원에서 선형적 구분을 시도합니다.  
일반적으로 모든 샘플을 랜드마크로 설정하여 샘플이 너무 많은 경우 연산량이 매우 많아질 수 있습니다.  

#### __2.3 계산 복잡도__
`linearSVC, SGD`는 커널트릭 없이 최적화로 $O(m*n)$의 복잡도를 가집니다.  
`SVC`의 커널은 $O(m^{2}*n)$ ~ $O(m^{3}*n)$ 사이로, 샘플이 많아지면 연산량이 급증합니다.  

### __3. SVM 회귀__

분류와 반대로, 마진 속에 가장 많이 샘플이 속하게 초평면을 구해 회귀직선으로 사용합니다.  
> `sklearn.svm.SVR`  
 
분류와 동일하게 비선형을 위해 커널을 사용할 수 있으며, 계산복잡도 역시 동일합니다.  
 

### __4. SVC의 수학적 원리__

#### __4.1 결정함수__ 


$\mathbf{w}^{T}\mathbf{x}+b$  

결정함수는 곧 초평면으로, 0이상을 양성으로 예측합니다.  
마진은 초평면으로부터 $\pm{1}$인 구역입니다.  
결정함수의 기울기는 가중치의 노름과 동일하며, 가중치가 낮을수록 마진은 넓어집니다.   



#### __4.2 목적함수__


$\lVert \mathbf{w} \rVert$를 최소화하는 함수입니다.  
또한 이상적으로 샘플이 마진 밖에 있어야 함으로 
$t_{i}=\begin{cases} 
1 \qquad posituve \\
-1 \quad \; negative
\end{cases}$
일 때, 아래와 같은 제약조건을 가진다.  
> $t_{i}(\mathbf{w}^{T}\mathbf{x}+b)\ge 1$

__하드마진__  
> $minimize\; {1\over{2}} \mathbf{w}^{T}\mathbf{w}$  
`[when]` $t_{i}(\mathbf{w}^{T}\mathbf{x}+b)\ge 1$

미분을 고려하여 노름의 제곱과 나누기 2를 최소화합니다.  


__소프트마진__ 
> $minimize\; {1\over{2}} \mathbf{w}^{T}\mathbf{w} + C{\sum_{i}^{m}{\zeta_i}}$  
`[when]` $t_{i}(\mathbf{w}^{T}\mathbf{x}+b)\ge 1-\zeta_{i} \; and \; \zeta_{i} \ge 0$

$\zeta$는 <b>슬랙변수</b>로, 높을수록 샘플이 마진을 많이 위반하여 마진에러가 늘어납니다.  
C는 w의 노름과 슬랙변수간의 트레이드오프를 결정합니다.  



#### __4.3 최적화__

조건이 있는 최적화 문제는 <b>쿼드라틱 프로그래밍</b>을 통해 해결할 수 있습니다.  
이것은 원리대로 해결하는 방식으로, 연산이 복잡하여 잘 쓰이지 않습니다.  

<b>쌍대문제</b>로 변환하면 샘플이 많은 경우 유리하고, 커널트릭을 사용하여 연산 효율적입니다.     
SVM의 경우 쌍대문제의 해가 원문제와 동일합니다.  

<b>커널트릭</b>은 더 높은 차원으로 매핑하는 방법으로, 연산을 간단하게 하기 위하여 사용됩니다.  
<b>머서의 정리</b>에 의하여 커널이 존재가 보장됨으로 구체적인 함수를 몰라도 사용할 수 있습니다.  
> 선형, 다항식, RBF, 시그모이드(하이퍼볼릭 탄젠트) 등이 있습니다.  

<span style='color:gray; font-size:70%'>_자세한 수식은 후에 더 공부하여 추가하겠습니다._</span>